In [1]:
conda install yfinance pyportfolioopt

Retrieving notices: ...working... done
Channels:
 - rapidsai
 - nvidia
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pyportfolioopt
    - yfinance


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    appdirs-1.4.4              |     pyh9f0ad1d_0          13 KB  conda-forge
    clarabel-0.7.1             |  py310h8c3a010_0         996 KB  conda-forge
    conda-24.7.1               |  py310hff52083_0         940 KB  conda-forge
    cvxopt-1.3.2               |  py310h7b0674a_2         496 KB  conda-forge
    cvxpy-1.5.2                |  py310hff52083_1         128 KB  conda-forge
    cvxpy-base-1.5.2           |  py310hf9f9076_1         1.2 MB  conda-forge
    dsdp-5.8                   | 

pip install ortools">=9.7"

In [2]:

import yfinance as yf
import pandas as pd
from pypfopt.efficient_frontier import EfficientFrontier
from  pypfopt import expected_returns, risk_models
from pypfopt import plotting
2

(CVXPY) Aug 17 01:27:15 PM: Encountered unexpected exception importing solver GLOP:
RuntimeError('Version of ortools (9.4.1874) is too old. Expected >= 9.5.0.')
(CVXPY) Aug 17 01:27:15 PM: Encountered unexpected exception importing solver PDLP:
RuntimeError('Version of ortools (9.4.1874) is too old. Expected >= 9.7.0.')


2

In [3]:
sp500 = pd.read_html('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol']


In [4]:

sp500.tolist()[:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

In [5]:

dfr = yf.download(
    sp500.tolist(), 
    start='2020-1-1',
    end='2024-7-28')['Adj Close']
dfr.shape

[**********************60%%***                   ]  300 of 503 completed

$BF.B: possibly delisted; No price data found  (1d 2020-1-1 -> 2024-7-28)


[*********************100%%**********************]  503 of 503 completed


(1149, 503)

In [6]:

df=dfr.dropna(axis='rows', thresh=50).dropna(axis='columns', how='any')
df.shape

(1149, 491)

In [7]:

df.sample(3)

Ticker,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-21,126.706787,17.420000,190.705292,138.393005,111.962158,82.269997,309.795746,520.229980,188.514175,79.788521,...,230.331543,33.411594,106.061020,62.765457,99.554657,112.410179,134.912659,141.059906,306.459991,181.705841
2020-02-18,82.222473,28.629999,77.584991,77.173187,82.190300,47.889999,200.197327,378.850006,109.100410,39.875332,...,199.679489,25.308500,130.670761,62.133785,48.050560,83.420570,96.148544,149.362579,236.660004,138.897659
2022-03-29,136.049362,18.160000,176.494034,147.415771,114.558838,48.560001,327.786865,466.329987,163.274582,83.227127,...,234.739288,35.898830,80.713120,66.639961,75.521065,86.798882,117.082687,127.240936,438.100006,188.044403


In [8]:
risk_models.cov_to_corr(risk_models.sample_cov(df))['KLAC'].sort_values()


Ticker
KR      0.067907
CPB     0.072098
SJM     0.083144
GIS     0.102537
CAG     0.115547
          ...   
MCHP    0.801538
TER     0.817315
AMAT    0.903322
LRCX    0.907746
KLAC    1.000000
Name: KLAC, Length: 491, dtype: float64

In [9]:
syms=df.columns.to_list()

In [10]:

mu = expected_returns.mean_historical_return(df[list(syms)])
S = risk_models.sample_cov(df[list(syms)])
mu.mean(), mu.max()

(0.10992350010952401, 1.1119690464358185)